In [74]:
import tensorflow as tf
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
from tensorflow.python.platform import tf_logging as logging
from tensorflow.contrib import slim
from nets import inception
from preprocessing import inception_preprocessing

import os
import time

In [75]:
#================ DATASET INFORMATION ======================
#State dataset directory where the tfrecord files are located
dataset_dir = './notMNIST'

#State where your log file is at. If it doesn't exist, create it.
log_dir = './logs'

#State where your checkpoint file is
checkpoint_file = './checkpoints/inception_v1.ckpt'

#State the image size you're resizing your images to. We will use the default inception size of 299.
image_size = inception.inception_v1.default_image_size

#State the number of classes to predict:
num_classes = 10

#State the labels file and read it
labels_file = './notMNIST/labels.txt'
labels = open(labels_file, 'r')

#Create a dictionary to refer each label to their string name
labels_to_name = {}
for line in labels:
    label, string_name = line.split(':')
    string_name = string_name[:-1] #Remove newline
    labels_to_name[int(label)] = string_name

#Create the file pattern of your TFRecord files so that it could be recognized later on
file_pattern = 'MNIST_%s_*.tfrecord'

#Create a dictionary that will help people understand your dataset better. This is required by the Dataset class later.
items_to_descriptions = {
    'image': 'A 3-channel RGB coloured flower image that is either tulips, sunflowers, roses, dandelion, or daisy.',
    'label': 'A label that is as such -- 0:daisy, 1:dandelion, 2:roses, 3:sunflowers, 4:tulips'
}

In [76]:
#================= TRAINING INFORMATION ==================
#State the number of epochs to train
num_epochs = 10

#State your batch size
batch_size = 10

#Learning rate information and configuration (Up to you to experiment)
initial_learning_rate = 0.0002
learning_rate_decay_factor = 0.7
num_epochs_before_decay = 2

In [77]:

def get_split(split_name, dataset_dir, file_pattern=file_pattern):
    """
    Obtains the split - training or validation - to create a Dataset class for feeding the examples into a queue later
    on. This function will set up the decoder and dataset information all into one Dataset class so that you can avoid
    the brute work later on.
    
    Your file_pattern is very important in locating the files later. 

    INPUTS:
        - split_name(str): 'train' or 'validation'. Used to get the correct data split of tfrecord files
        - dataset_dir(str): the dataset directory where the tfrecord files are located
        - file_pattern(str): the file name structure of the tfrecord files in order to get the correct data

    OUTPUTS:
    - dataset (Dataset): A Dataset class object where we can read its various components for easier batch creation.
    """
    #First check whether the split_name is train or validation
    if split_name not in ['train', 'validation']:
        raise ValueError(
            'The split_name %s is not recognized. Please input either train or validation as the split_name'%(split_name))

    #Create the full path for a general file_pattern to locate the tfrecord_files
    file_pattern_path = os.path.join(dataset_dir, file_pattern % (split_name))

    #Count the total number of examples in all of these shard
    num_samples = 0
    file_pattern_for_counting = 'MNIST_' + split_name
    tfrecords_to_count = [os.path.join(dataset_dir, file) for file in os.listdir(dataset_dir)
                          if file.startswith(file_pattern_for_counting)]
    
    for tfrecord_file in tfrecords_to_count:
        for record in tf.python_io.tf_record_iterator(tfrecord_file):
            num_samples += 1

    #Create a reader, which must be a TFRecord reader in this case
    reader = tf.TFRecordReader

    #Create the keys_to_features dictionary for the decoder
    keys_to_features = {
      'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
      'image/format': tf.FixedLenFeature((), tf.string, default_value='jpg'),
      'image/class/label': tf.FixedLenFeature(
          [], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
    }

    #Create the items_to_handlers dictionary for the decoder.
    items_to_handlers = {
    'image': slim.tfexample_decoder.Image(),
    'label': slim.tfexample_decoder.Tensor('image/class/label'),
    }

    #Start to create the decoder
    decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)

    #Create the labels_to_name file
    labels_to_name_dict = labels_to_name

    #Actually create the dataset
    dataset = slim.dataset.Dataset(
        data_sources = file_pattern_path,
        decoder = decoder,
        reader = reader,
        num_readers = 4,
        num_samples = num_samples,
        num_classes = num_classes,
        labels_to_name = labels_to_name_dict,
        items_to_descriptions = items_to_descriptions)

    return dataset

In [78]:
def load_batch(dataset, batch_size, height=image_size, width=image_size, is_training=True):
    '''
    Loads a batch for training.

    INPUTS:
    - dataset(Dataset): a Dataset class object that is created from the get_split function
    - batch_size(int): determines how big of a batch to train
    - height(int): the height of the image to resize to during preprocessing
    - width(int): the width of the image to resize to during preprocessing
    - is_training(bool): to determine whether to perform a training or evaluation preprocessing

    OUTPUTS:
    - images(Tensor): a Tensor of the shape (batch_size, height, width, channels) that contain one batch of images
    - labels(Tensor): the batch's labels with the shape (batch_size,) (requires one_hot_encoding).

    '''
    #First create the data_provider object
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset,
        common_queue_capacity = 24 + 3 * batch_size,
        common_queue_min = 24)

    #Obtain the raw image using the get method
    raw_image, label = data_provider.get(['image', 'label'])

    #Perform the correct preprocessing for this image depending if it is training or evaluating
    image = inception_preprocessing.preprocess_image(raw_image, height, width, is_training)

    #As for the raw images, we just do a simple reshape to batch it up
    raw_image = tf.expand_dims(raw_image, 0)
    raw_image = tf.image.resize_nearest_neighbor(raw_image, [height, width])
    raw_image = tf.squeeze(raw_image)

    #Batch up the image by enqueing the tensors internally in a FIFO queue and dequeueing many elements with tf.train.batch.
    images, raw_images, labels = tf.train.batch(
        [image, raw_image, label],
        batch_size = batch_size,
        num_threads = 4,
        capacity = 4 * batch_size,
        allow_smaller_final_batch = True)

    return images, raw_images, labels


In [79]:
myData = get_split('train', dataset_dir, file_pattern=file_pattern)
steps_per_epoch = myData.num_samples / batch_size
print("training set size:\t", myData.num_samples)
print("number of epochs:\t", num_epochs)
print("number of steps per epochs:\t", steps_per_epoch)
print("total number of steps:\t", steps_per_epoch * num_epochs, "\n===============================\n")

if batch_size != 10:
    print("estimated time per step:\t", (1+abs(10/batch_size))*0.25, " sec")
    print("estimated time to train:\t", (steps_per_epoch * num_epochs * (1+abs(10/batch_size))*0.25),
         " secs")
    print("estimated time to train:\t", (steps_per_epoch * num_epochs * (1+abs(10/batch_size))*0.25 / 60),
         " minutes")
    print("estimated time to train:\t", (steps_per_epoch * num_epochs * (1+abs(10/batch_size))*0.25 / (60*60)),
         " hours")
else:
    print("estimated time per step:\t", 0.25 , " sec")
    print("estimated time to train:\t", (steps_per_epoch * num_epochs * 0.25), " secs")
    print("estimated time to train:\t", (steps_per_epoch * num_epochs * 0.25 / 60), " minutes")
    print("estimated time to train:\t", (steps_per_epoch * num_epochs * 0.25 / (60*60)), " hours")

training set size:	 13107
number of epochs:	 10
number of steps per epochs:	 1310.7
total number of steps:	 13107.0 

estimated time per step:	 0.25  sec
estimated time to train:	 3276.75  secs
estimated time to train:	 54.6125  minutes
estimated time to train:	 0.9102083333333333  hours


In [80]:
def run():
    #Create the log directory here. Must be done here otherwise import will activate this unneededly.
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)
    
    with tf.Graph().as_default() as graph:
        tf.logging.set_verbosity(tf.logging.INFO) #Set the verbosity to INFO level
        
        #First create the dataset and load one batch
        dataset = get_split('train', dataset_dir, file_pattern=file_pattern)
        images, _, labels = load_batch(dataset, batch_size=batch_size)
        
        #Know the number steps to take before decaying the learning rate and batches per epoch
        num_batches_per_epoch = dataset.num_samples / batch_size
        num_steps_per_epoch = num_batches_per_epoch #Because one step is one batch processed
        decay_steps = int(num_epochs_before_decay * num_steps_per_epoch)
        
        #Create the model inference
        with slim.arg_scope(inception.inception_v1_arg_scope()):
            logits, end_points = inception.inception_v1(
                images,
                num_classes = dataset.num_classes,
                is_training = True)
            
            #Define the scopes that you want to exclude for restoration
            #exclude = ['InceptionResnetV1/Logits', 'InceptionResnetV1/AuxLogits']
            exclude = ["InceptionV1/Logits", "InceptionV1/AuxLogits"]
            variables_to_restore = slim.get_variables_to_restore(exclude = exclude)
            
            #Perform one-hot-encoding of the labels (Try one-hot-encoding within the load_batch function!)
            one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)
            
            #Performs the equivalent to tf.nn.sparse_softmax_cross_entropy_with_logits but enhanced with checks
            loss = tf.losses.softmax_cross_entropy(onehot_labels = one_hot_labels, logits = logits)
            total_loss = tf.losses.get_total_loss()    #obtain the regularization losses as well
            
            #Create the global step for monitoring the learning_rate and training.
            global_step = get_or_create_global_step()
            
            #Define your exponentially decaying learning rate
            lr = tf.train.exponential_decay(
                learning_rate = initial_learning_rate,
                global_step = global_step,
                decay_steps = decay_steps,
                decay_rate = learning_rate_decay_factor,
                staircase = True)
            
            #Now we can define the optimizer that takes on the learning rate
            optimizer = tf.train.AdamOptimizer(learning_rate = lr)
            
            #Create the train_op.
            train_op = slim.learning.create_train_op(total_loss, optimizer)
            
            #State the metrics that you want to predict. We get a predictions that is not one_hot_encoded.
            predictions = tf.argmax(end_points['Predictions'], 1)
            probabilities = end_points['Predictions']
            accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, labels)
            metrics_op = tf.group(accuracy_update)
            
            #Now finally create all the summaries you need to monitor and group them into one summary op.
            tf.summary.scalar('losses/Total_Loss', total_loss)
            tf.summary.scalar('accuracy', accuracy)
            tf.summary.scalar('learning_rate', lr)
            my_summary_op = tf.summary.merge_all()
            
            def train_step(sess, train_op, global_step):
                '''
                Simply runs a session for the three arguments provided and gives a logging on the time elapsed
                for each global step
                '''
                #Check the time for each sess run
                start_time = time.time()
                total_loss, global_step_count, _ = sess.run([train_op, global_step, metrics_op])
                time_elapsed = time.time() - start_time
                
                #Run the logging to print some results
                logging.info('global step %s: loss: %.4f (%.2f sec/step)', global_step_count, total_loss, time_elapsed)
                return total_loss, global_step_count
            
            #Now we create a saver function that actually restores the variables from a checkpoint file in a sess
            saver = tf.train.Saver(variables_to_restore)
            def restore_fn(sess):
                return saver.restore(sess, checkpoint_file)
            
            #Define your supervisor for running a managed session. Do not run the summary_op automatically or else
            #it will consume too much memory
            sv = tf.train.Supervisor(logdir = log_dir, summary_op = None, init_fn = restore_fn)
            
            with sv.managed_session() as sess:
                for step in range(int(num_steps_per_epoch * num_epochs)):
                    #At the start of every epoch, show the vital information:
                    if step % num_batches_per_epoch == 0:
                        logging.info('Epoch %s/%s', step/num_batches_per_epoch + 1, num_epochs)
                        learning_rate_value, accuracy_value = sess.run([lr, accuracy])
                        logging.info('Current Learning Rate: %s', learning_rate_value)
                        logging.info('Current Streaming Accuracy: %s', accuracy_value)
                        
                        # optionally, print your logits and predictions for a sanity check that things are going fine.
                        logits_value, probabilities_value, predictions_value, labels_value = sess.run([logits,
                                                                                                       probabilities,
                                                                                                       predictions,
                                                                                                       labels])
                        print ('logits: \n', logits_value)
                        print ('Probabilities: \n', probabilities_value)
                        print ('predictions: \n', predictions_value)
                        print ('Labels:\n', labels_value  )                
                        
                    #Log the summaries every 10 step.
                    if step % 10 == 0:
                        loss, _ = train_step(sess, train_op, sv.global_step)
                        summaries = sess.run(my_summary_op)
                        sv.summary_computed(sess, summaries)
                            
                    #If not, simply run the training step
                    else:
                        loss, _ = train_step(sess, train_op, sv.global_step)
                            
                #We log the final training loss and accuracy
                logging.info('Final Loss: %s', loss)
                logging.info('Final Accuracy: %s', sess.run(accuracy))
                
                #Once all the training has been done, save the log files and checkpoint model
                logging.info('Finished training! Saving model to disk now.')
                sv.saver.save(sess, sv.save_path, global_step = sv.global_step)

In [81]:
run()

INFO:tensorflow:Restoring parameters from ./logs\model.ckpt-5242
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path ./logs\model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Epoch 1.0/10
INFO:tensorflow:Current Learning Rate: 9.8e-05
INFO:tensorflow:Current Streaming Accuracy: 0.0
logits: 
 [[-0.34271356  0.29572096 -2.96835804  1.67822361 -0.35230684 -0.73520589
  -2.5443759   0.77952355  0.67852443  0.1368531 ]
 [ 2.12685227 -2.15678716  2.60337353 -1.73825455  1.55357504 -0.5635159
   3.30852103 -2.54855561 -2.47960258 -3.39283347]
 [-2.64832091 -0.52161407 -0.70238131  0.4492695  -0.34372246 -0.51924837
  -1.23693085 -1.91372228  1.86497843  2.71209931]
 [ 0.4709067  -1.69407642  2.39326262 -1.03554726 -0.20659271 -0.72757947
   1.00006151 -1.11668932 -0.65916729  1.3194828 ]
 [ 1.59953535 -0.39665684 -4.1407032  -1.30807519  0.03717129 -0.1130725
   5.01292133  0.54816419 -3.85354924 -4.67212439]
 [-0.26417616  5.98967075 -4.45662

INFO:tensorflow:global step 5319: loss: 1.1147 (0.24 sec/step)
INFO:tensorflow:global step 5320: loss: 1.0228 (0.23 sec/step)
INFO:tensorflow:global step 5321: loss: 1.3288 (0.23 sec/step)
INFO:tensorflow:global step 5322: loss: 1.5071 (0.22 sec/step)
INFO:tensorflow:global step 5323: loss: 1.1058 (0.25 sec/step)
INFO:tensorflow:global step 5324: loss: 0.8486 (0.25 sec/step)
INFO:tensorflow:global step 5325: loss: 1.0894 (0.23 sec/step)
INFO:tensorflow:global step 5326: loss: 1.2386 (0.25 sec/step)
INFO:tensorflow:global step 5327: loss: 1.0123 (0.26 sec/step)
INFO:tensorflow:global step 5328: loss: 0.9198 (0.24 sec/step)
INFO:tensorflow:global step 5329: loss: 0.9725 (0.25 sec/step)
INFO:tensorflow:global step 5330: loss: 1.6459 (0.25 sec/step)
INFO:tensorflow:global step 5331: loss: 1.2975 (0.23 sec/step)
INFO:tensorflow:global step 5332: loss: 0.9278 (0.25 sec/step)
INFO:tensorflow:global step 5333: loss: 0.9840 (0.25 sec/step)
INFO:tensorflow:global step 5334: loss: 0.8279 (0.23 se

INFO:tensorflow:global step 5450: loss: 0.8983 (0.24 sec/step)
INFO:tensorflow:global step 5451: loss: 0.9395 (0.25 sec/step)
INFO:tensorflow:global step 5452: loss: 0.6754 (0.24 sec/step)
INFO:tensorflow:global step 5453: loss: 0.6375 (0.24 sec/step)
INFO:tensorflow:global step 5454: loss: 0.8087 (0.22 sec/step)
INFO:tensorflow:global step 5455: loss: 1.1642 (0.25 sec/step)
INFO:tensorflow:global step 5456: loss: 0.7770 (0.22 sec/step)
INFO:tensorflow:global step 5457: loss: 1.4081 (0.24 sec/step)
INFO:tensorflow:global step 5458: loss: 1.0720 (0.23 sec/step)
INFO:tensorflow:global step 5459: loss: 1.6606 (0.25 sec/step)
INFO:tensorflow:global step 5460: loss: 1.3189 (0.24 sec/step)
INFO:tensorflow:global step 5461: loss: 0.7883 (0.23 sec/step)
INFO:tensorflow:global step 5462: loss: 0.5604 (0.25 sec/step)
INFO:tensorflow:global step 5463: loss: 0.5584 (0.23 sec/step)
INFO:tensorflow:global step 5464: loss: 1.2654 (0.23 sec/step)
INFO:tensorflow:global step 5465: loss: 1.5560 (0.24 se

INFO:tensorflow:global step 5581: loss: 1.0355 (0.24 sec/step)
INFO:tensorflow:global step 5582: loss: 2.4753 (0.24 sec/step)
INFO:tensorflow:global step 5583: loss: 0.7210 (0.23 sec/step)
INFO:tensorflow:global step 5584: loss: 0.6255 (0.24 sec/step)
INFO:tensorflow:global step 5585: loss: 1.1996 (0.23 sec/step)
INFO:tensorflow:global step 5586: loss: 1.0685 (0.23 sec/step)
INFO:tensorflow:global step 5587: loss: 0.9047 (0.23 sec/step)
INFO:tensorflow:global step 5588: loss: 0.5475 (0.24 sec/step)
INFO:tensorflow:global step 5589: loss: 0.9463 (0.23 sec/step)
INFO:tensorflow:global step 5590: loss: 0.8208 (0.23 sec/step)
INFO:tensorflow:global step 5591: loss: 0.8563 (0.23 sec/step)
INFO:tensorflow:global step 5592: loss: 1.2677 (0.23 sec/step)
INFO:tensorflow:global step 5593: loss: 1.2900 (0.24 sec/step)
INFO:tensorflow:global step 5594: loss: 0.7310 (0.23 sec/step)
INFO:tensorflow:global step 5595: loss: 1.4769 (0.23 sec/step)
INFO:tensorflow:global step 5596: loss: 1.6397 (0.24 se

INFO:tensorflow:global step 5712: loss: 0.9944 (0.24 sec/step)
INFO:tensorflow:global step 5713: loss: 1.3082 (0.23 sec/step)
INFO:tensorflow:global step 5714: loss: 1.3399 (0.23 sec/step)
INFO:tensorflow:global step 5715: loss: 1.0438 (0.24 sec/step)
INFO:tensorflow:global step 5716: loss: 0.8990 (0.23 sec/step)
INFO:tensorflow:global step 5717: loss: 1.1687 (0.25 sec/step)
INFO:tensorflow:global step 5718: loss: 1.4382 (0.22 sec/step)
INFO:tensorflow:global step 5719: loss: 0.6650 (0.23 sec/step)
INFO:tensorflow:global step 5720: loss: 1.0383 (0.23 sec/step)
INFO:tensorflow:global step 5721: loss: 0.6951 (0.24 sec/step)
INFO:tensorflow:global step 5722: loss: 1.6224 (0.23 sec/step)
INFO:tensorflow:global step 5723: loss: 0.6148 (0.23 sec/step)
INFO:tensorflow:global step 5724: loss: 0.8812 (0.24 sec/step)
INFO:tensorflow:global step 5725: loss: 1.3124 (0.23 sec/step)
INFO:tensorflow:global step 5726: loss: 1.7448 (0.23 sec/step)
INFO:tensorflow:global step 5727: loss: 0.9884 (0.24 se

...

INFO:tensorflow:global step 17769: loss: 1.0526 (0.29 sec/step)
INFO:tensorflow:global step 17770: loss: 0.6133 (0.25 sec/step)
INFO:tensorflow:global step 17771: loss: 0.5980 (0.24 sec/step)
INFO:tensorflow:global step 17772: loss: 0.6855 (0.23 sec/step)
INFO:tensorflow:global step 17773: loss: 0.8698 (0.23 sec/step)
INFO:tensorflow:global step 17774: loss: 0.3952 (0.24 sec/step)
INFO:tensorflow:global step 17775: loss: 0.8200 (0.23 sec/step)
INFO:tensorflow:global step 17776: loss: 1.0285 (0.25 sec/step)
INFO:tensorflow:global step 17777: loss: 1.1980 (0.22 sec/step)
INFO:tensorflow:global step 17778: loss: 0.6813 (0.25 sec/step)
INFO:tensorflow:global step 17779: loss: 0.6149 (0.23 sec/step)
INFO:tensorflow:global step 17780: loss: 0.6224 (0.24 sec/step)
INFO:tensorflow:global step 17781: loss: 0.5605 (0.23 sec/step)
INFO:tensorflow:global step 17782: loss: 0.9145 (0.29 sec/step)
INFO:tensorflow:global step 17783: loss: 0.5551 (0.28 sec/step)
INFO:tensorflow:global step 17784: loss:

INFO:tensorflow:global step 17898: loss: 1.4903 (0.23 sec/step)
INFO:tensorflow:global step 17899: loss: 0.4873 (0.23 sec/step)
INFO:tensorflow:global step 17900: loss: 0.9032 (0.24 sec/step)
INFO:tensorflow:global step 17901: loss: 0.3010 (0.29 sec/step)
INFO:tensorflow:global step 17902: loss: 1.2118 (0.26 sec/step)
INFO:tensorflow:global step 17903: loss: 0.9367 (0.24 sec/step)
INFO:tensorflow:global step 17904: loss: 0.8082 (0.24 sec/step)
INFO:tensorflow:global step 17905: loss: 0.9384 (0.23 sec/step)
INFO:tensorflow:global step 17906: loss: 1.6473 (0.25 sec/step)
INFO:tensorflow:global step 17907: loss: 0.7109 (0.25 sec/step)
INFO:tensorflow:global step 17908: loss: 1.3398 (0.26 sec/step)
INFO:tensorflow:global step 17909: loss: 1.0471 (0.26 sec/step)
INFO:tensorflow:global step 17910: loss: 1.1208 (0.23 sec/step)
INFO:tensorflow:global step 17911: loss: 0.7190 (0.24 sec/step)
INFO:tensorflow:global step 17912: loss: 0.8040 (0.24 sec/step)
INFO:tensorflow:global step 17913: loss:

INFO:tensorflow:global step 18027: loss: 0.4404 (0.24 sec/step)
INFO:tensorflow:global step 18028: loss: 1.2741 (0.23 sec/step)
INFO:tensorflow:global step 18029: loss: 0.6626 (0.23 sec/step)
INFO:tensorflow:global step 18030: loss: 0.4077 (0.24 sec/step)
INFO:tensorflow:global step 18031: loss: 0.7949 (0.23 sec/step)
INFO:tensorflow:global step 18032: loss: 0.7972 (0.24 sec/step)
INFO:tensorflow:global step 18033: loss: 0.6019 (0.23 sec/step)
INFO:tensorflow:global step 18034: loss: 0.8515 (0.23 sec/step)
INFO:tensorflow:global step 18035: loss: 0.6042 (0.24 sec/step)
INFO:tensorflow:global step 18036: loss: 1.2655 (0.26 sec/step)
INFO:tensorflow:global step 18037: loss: 1.0831 (0.27 sec/step)
INFO:tensorflow:global step 18038: loss: 0.7749 (0.23 sec/step)
INFO:tensorflow:global step 18039: loss: 0.4307 (0.24 sec/step)
INFO:tensorflow:global step 18040: loss: 0.5503 (0.23 sec/step)
INFO:tensorflow:global step 18041: loss: 1.0866 (0.22 sec/step)
INFO:tensorflow:global step 18042: loss:

INFO:tensorflow:global step 18156: loss: 0.3599 (0.23 sec/step)
INFO:tensorflow:global step 18157: loss: 0.5600 (0.23 sec/step)
INFO:tensorflow:global step 18158: loss: 0.6248 (0.22 sec/step)
INFO:tensorflow:global step 18159: loss: 1.1910 (0.23 sec/step)
INFO:tensorflow:global step 18160: loss: 0.7771 (0.23 sec/step)
INFO:tensorflow:global step 18161: loss: 0.4042 (0.24 sec/step)
INFO:tensorflow:global step 18162: loss: 0.6895 (0.23 sec/step)
INFO:tensorflow:global step 18163: loss: 0.4472 (0.25 sec/step)
INFO:tensorflow:global step 18164: loss: 2.0033 (0.24 sec/step)
INFO:tensorflow:global step 18165: loss: 0.4955 (0.23 sec/step)
INFO:tensorflow:global step 18166: loss: 0.8593 (0.23 sec/step)
INFO:tensorflow:global step 18167: loss: 0.9115 (0.22 sec/step)
INFO:tensorflow:global step 18168: loss: 0.5435 (0.25 sec/step)
INFO:tensorflow:global step 18169: loss: 1.0798 (0.24 sec/step)
INFO:tensorflow:global step 18170: loss: 1.2593 (0.23 sec/step)
INFO:tensorflow:global step 18171: loss:

INFO:tensorflow:global step 18285: loss: 0.6783 (0.25 sec/step)
INFO:tensorflow:global step 18286: loss: 0.4963 (0.28 sec/step)
INFO:tensorflow:global step 18287: loss: 0.4326 (0.25 sec/step)
INFO:tensorflow:global step 18288: loss: 0.6402 (0.22 sec/step)
INFO:tensorflow:global step 18289: loss: 0.8164 (0.23 sec/step)
INFO:tensorflow:global step 18290: loss: 1.4376 (0.23 sec/step)
INFO:tensorflow:global step 18291: loss: 1.8031 (0.24 sec/step)
INFO:tensorflow:global step 18292: loss: 0.8610 (0.23 sec/step)
INFO:tensorflow:global step 18293: loss: 1.2235 (0.25 sec/step)
INFO:tensorflow:global step 18294: loss: 0.9520 (0.24 sec/step)
INFO:tensorflow:global step 18295: loss: 0.6777 (0.23 sec/step)
INFO:tensorflow:global step 18296: loss: 0.9660 (0.25 sec/step)
INFO:tensorflow:global step 18297: loss: 0.7184 (0.23 sec/step)
INFO:tensorflow:global step 18298: loss: 1.1724 (0.25 sec/step)
INFO:tensorflow:global step 18299: loss: 0.3340 (0.25 sec/step)
INFO:tensorflow:global step 18300: loss: